<a href="https://colab.research.google.com/github/Aarushi-29/Fake_news_detection/blob/main/scraping_proj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install feedparser


!pip install newspaper3k

In [ ]:
with open('NewsPapers.json') as data_file:
    companies = json.load(data_file)

In [ ]:
import feedparser as fp
import json
import newspaper
from newspaper import Article
from time import mktime
from datetime import datetime

# Set the limit for number of articles to download
LIMIT = 14500

data = {}
data['newspapers'] = {}

# Loads the JSON files with news sites
with open('NewsPapers.json') as data_file:
    companies = json.load(data_file)

count = 1

# Iterate through each news company
for company, value in companies.items():
    # If a RSS link is provided in the JSON file, this will be the first choice.
    # Reason for this is that, RSS feeds often give more consistent and correct data.
    # If you do not want to scrape from the RSS-feed, just leave the RSS attr empty in the JSON file.
    if 'rss' in value:
        d = fp.parse(value['rss'])
        print("Downloading articles from ", company)
        newsPaper = {
            "rss": value['rss'],
            "link": value['link'],
            "articles": []
        }
        for entry in d.entries:
            # Check if publish date is provided, if no the article is skipped.
            # This is done to keep consistency in the data and to keep the script from crashing.
            if hasattr(entry, 'published'):
                if count > LIMIT:
                    break
                article = {}
                article['link'] = entry.link
                date = entry.published_parsed
                article['published'] = datetime.fromtimestamp(mktime(date)).isoformat()
                try:
                    content = Article(entry.link)
                    content.download()
                    content.parse()
                except Exception as e:
                    # If the download for some reason fails (ex. 404) the script will continue downloading
                    # the next article.
                    print(e)
                    print("continuing...")
                    continue
                article['title'] = content.title
                article['text'] = content.text
                newsPaper['articles'].append(article)
                print(count, "articles downloaded from", company, ", url: ", entry.link)
                count = count + 1
    else:
        # This is the fallback method if a RSS-feed link is not provided.
        # It uses the python newspaper library to extract articles
        print("Building site for ", company)
        paper = newspaper.build(value['link'], memoize_articles=False)
        newsPaper = {
            "link": value['link'],
            "articles": []
        }
        noneTypeCount = 0
        for content in paper.articles:
            if count > LIMIT:
                break
            try:
                content.download()
                content.parse()
            except Exception as e:
                print(e)
                print("continuing...")
                continue
            # Again, for consistency, if there is no found publish date the article will be skipped.
            # After 10 downloaded articles from the same newspaper without publish date, the company will be skipped.
            if content.publish_date is None:
                print(count, " Article has date of type None...")
                noneTypeCount = noneTypeCount + 1
                if noneTypeCount > 100:
                    print("Too many noneType dates, aborting...")
                    noneTypeCount = 0
                    break
                count = count + 1
                continue
            article = {}
            article['title'] = content.title
            article['text'] = content.text
            article['link'] = content.url
            article['published'] = content.publish_date.isoformat()
            newsPaper['articles'].append(article)
            print(count, "articles downloaded from", company, " using newspaper, url: ", content.url)
            count = count + 1
            noneTypeCount = 0
    count = 1
    data['newspapers'][company] = newsPaper

# Finally it saves the articles as a JSON-file.
try:
    with open('scraped_articles.json', 'w') as outfile:
        json.dump(data, outfile)
except Exception as e: print(e)

1 articles downloaded from bbc , url:  https://www.bbc.co.uk/news/uk-england-merseyside-65960514?at_medium=RSS&at_campaign=KARANGA
2 articles downloaded from bbc , url:  https://www.bbc.co.uk/news/uk-politics-66547111?at_medium=RSS&at_campaign=KARANGA
3 articles downloaded from bbc , url:  https://www.bbc.co.uk/news/world-us-canada-66499946?at_medium=RSS&at_campaign=KARANGA
4 articles downloaded from bbc , url:  https://www.bbc.co.uk/news/world-africa-66540039?at_medium=RSS&at_campaign=KARANGA
5 articles downloaded from bbc , url:  https://www.bbc.co.uk/news/uk-scotland-66547219?at_medium=RSS&at_campaign=KARANGA
6 articles downloaded from bbc , url:  https://www.bbc.co.uk/news/uk-66540732?at_medium=RSS&at_campaign=KARANGA
7 articles downloaded from bbc , url:  https://www.bbc.co.uk/news/business-66532694?at_medium=RSS&at_campaign=KARANGA
8 articles downloaded from bbc , url:  https://www.bbc.co.uk/sport/football/66541044?at_medium=RSS&at_campaign=KARANGA
9 articles downloaded from bbc 

In [ ]:
with open('scraped_articles.json') as json_data:
    d = json.load(json_data)


for i, site in enumerate((list(d['newspapers']))):
    print(i, site)

0 bbc
1 breitbart
2 cnn
3 foxnews
4 nytimes_frontpage
5 nytimes_international
6 theguardian
7 washingtonpost
8 wsj
9 timesofindia
10 thehindu
11 hindustantimes
12 indianexpress
13 dailypioneer
14 deccanherald
15 opindia


In [ ]:
import pandas as pd
for i, site in enumerate((list(d['newspapers']))):
    articles = list(d['newspapers'][site]['articles'])
    if i == 0:
        df = pd.DataFrame.from_dict(articles)
        df["site"] = site
    else:
        new_df = pd.DataFrame.from_dict(articles)
        new_df["site"] = site
        df = pd.concat([df, new_df], ignore_index = True)

In [ ]:
df.shape

(767, 5)

In [ ]:
df

,link,published,title,text,site
0,https://www.bbc.co.uk/news/uk-england-merseysi...,2023-08-18T14:42:27,Nurse Lucy Letby guilty of murdering seven bab...,The Department of Health said the inquiry woul...,bbc
1,https://www.bbc.co.uk/news/uk-politics-6654711...,2023-08-18T14:18:56,Foreign secretary James Cleverly to attend Wor...,The government has said it has no plans for a ...,bbc
2,https://www.bbc.co.uk/news/world-us-canada-664...,2023-08-18T14:02:40,Hurricane Hilary: Category 4 hurricane approac...,The National Weather Service (NWS) said the po...,bbc
3,https://www.bbc.co.uk/news/world-africa-665400...,2023-08-18T11:19:11,Ethiopia's Tigray crisis: Deaths from starvati...,WFP and USAid rallied to assist some six milli...,bbc
4,https://www.bbc.co.uk/news/uk-scotland-6654721...,2023-08-18T13:35:00,Recreating the face of Bonnie Prince Charlie,A model of the face of Bonnie Prince Charlie h...,bbc
...,...,...,...,...,...
762,https://www.opindia.com/2023/08/rbi-unveils-ud...,2023-08-18T12:45:10,RBI unveils UDGAM Portal: Know its importance ...,In a significant stride towards resolving the ...,opindia
763,https://www.opindia.com/2023/08/hardoi-maulana...,2023-08-18T12:35:00,"UP: Hardoi Maulana threatens Centre over UCC, ...","A cleric of Lal Pir Mosque of Gopamau, Hardoi ...",opindia
764,https://www.opindia.com/2023/08/seema-haidars-...,2023-08-18T12:18:05,Seema Haidar's ex-husband wants to come to Ind...,Seema Haider’s ex-husband from Pakistan identi...,opindia
765,https://www.opindia.com/2023/08/sushant-singh-...,2023-08-18T11:38:38,Sushant Singh Rajput's father moves Delhi HC a...,A Division Bench of Delhi High Court has sough...,opindia


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
!cp scraped_articles.json "/content/gdrive/My Drive/Fake_news_proj/"